In [ ]:
##
# @file This file is part of EDGE.
#
# @author Alexander Breuer (anbreuer AT ucsd.edu)
#
# @section LICENSE
# Copyright (c) 2016, Regents of the University of California
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:
#
# 1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
#
# 2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
#
# 3. Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
#
# @section DESCRIPTION
# Derivation of basis functions and the attached matrices for "rectangular" elements: Lines, quads, hexes.
##

In [ ]:
import sympy
import matplotlib.pyplot
sympy.init_printing(use_unicode=True)

In [ ]:
# upper polynomial degree
l_degUp = 3

# array of polynomials
l_pols = []

xib = sympy.symbols('xib') # coordinates of the basis [-1,1]
xi = sympy.symbols('xi') # mapped coordinates in [0,1]

# derive polynomials
for l_deg in xrange(0, l_degUp):
  # add legendre polynomials, shifted to interval [0,1] 
  l_pols    = l_pols + [ sympy.legendre(l_deg, xib).subs(xib, (2*xi-1)) ]

sympy.plotting.plot( l_pols[0], l_pols[1], l_pols[2], l_pols[3], l_pols[4], l_pols[5], (xi, 0, 1))

In [ ]:
# compute the mass matrix
l_mm = sympy.Matrix( l_pols ) * sympy.Matrix.transpose( sympy.Matrix( l_pols ) )
l_mm = sympy.integrate( l_mm, (xi, 0, 1) )

# compute the stiffness matrix
l_sm = sympy.Matrix( l_pols )
l_sm = l_sm * sympy.Matrix.transpose( sympy.Matrix( sympy.diff( l_sm ) ) )
l_sm = sympy.integrate( l_sm, (xi, 0, 1) )

l_mm, l_sm, l_sm * l_mm.inv(), l_sm.transpose() * l_mm.inv()

In [ ]:
# compute flux matrices

# evaluation of basis function at left side (0)
l_left = []
for l_deg in xrange(0, l_degUp):
  l_left = l_left + [l_pols[l_deg].subs(xi, 0)]

# evalution of basis functions at right side (1)

l_right = []
for l_deg in xrange(0, l_degUp):
  l_right = l_right + [l_pols[l_deg].subs(xi, 1)]

# compute local flux matrices
l_localFm = dict()
l_localFm['fM0'] = sympy.Matrix( l_left ) *  sympy.Matrix.transpose( sympy.Matrix( l_left ))

l_localFm['fM1'] = sympy.Matrix( l_right ) *  sympy.Matrix.transpose( sympy.Matrix( l_right ))

# compute neighboring flux matrices
l_neighFm = dict()

l_neighFm['fP00'] = sympy.Matrix( l_left )  *  sympy.Matrix.transpose( sympy.Matrix( l_left ))
l_neighFm['fP01'] = sympy.Matrix( l_right )  *  sympy.Matrix.transpose( sympy.Matrix( l_left ))
l_neighFm['fP10'] = sympy.Matrix( l_left ) *  sympy.Matrix.transpose( sympy.Matrix( l_right ))
l_neighFm['fP11'] = sympy.Matrix( l_right ) *  sympy.Matrix.transpose( sympy.Matrix( l_right ))

In [ ]:
# generate code for our basis functions
print "assert( b <", l_degUp,");"
for l_deg in xrange(0, l_degUp):
  print "if( b ==", l_deg, ") {" 
  print "  val    =", sympy.ccode(            l_pols[l_deg]  ).replace("pow", "std::pow"), ";"
  print "}"

print '*******************************'

# generate code for derivatives of our basis functions
print "assert( b <", l_degUp,");"
for l_deg in xrange(0, l_degUp):
  print "if( b ==", l_deg, ") {" 
  print "  valDxi =", sympy.ccode( sympy.diff(l_pols[l_deg]) ).replace("pow", "std::pow"), ";"
  print "}"


In [ ]:
from sympy.integrals import quadrature
quadrature.gauss_legendre( 3, 20 )

l_neighFm['fP00'] * l_mm.inv(), l_neighFm['fP01'] * l_mm.inv(), 

# Quads
Remark: This is not a hierarchical format since we go for xi first and then for eta!

In [ ]:
# build quad basis functions as tensor product
l_bQuads = []
eta = sympy.symbols('eta')
chi = sympy.symbols('chi')

for l_degEta in xrange(0, l_degUp):
  for l_degXi in xrange(0, l_degUp):
    l_bQuads = l_bQuads + [ l_pols[l_degXi] * l_pols[l_degEta].subs( xi, eta  ) ]

In [ ]:
import mpmath
# compute the mass matrix
l_mmQuads = sympy.Matrix( l_bQuads ) * sympy.Matrix.transpose( sympy.Matrix( l_bQuads ) )

for l_en in xrange(0, len(l_mmQuads)):
  l_mmQuads[l_en] = sympy.integrate( l_mmQuads[l_en], (xi, 0, 1), (eta, 0, 1) )

# compute the stiffness matrices
l_smQuads = {}
l_smQuads['xi']  = sympy.Matrix( l_bQuads )
l_smQuads['eta'] = sympy.Matrix( l_bQuads )

l_smQuads['xi']  = l_smQuads['xi']  * sympy.Matrix.transpose( sympy.Matrix( sympy.diff( l_smQuads['xi'], xi  ) ) )
l_smQuads['eta'] = l_smQuads['eta'] * sympy.Matrix.transpose( sympy.Matrix( sympy.diff( l_smQuads['eta'], eta ) ) )

for l_en in xrange(0, len(l_smQuads['xi'])):
  l_smQuads['xi'][l_en] = sympy.integrate( l_smQuads['xi'][l_en], (xi, 0, 1), (eta, 0, 1) )
  l_smQuads['eta'][l_en] = sympy.integrate( l_smQuads['eta'][l_en], (xi, 0, 1), (eta, 0, 1) )

In [ ]:
# compute flux matrices
l_localFm = dict()
l_neighFm = dict()

# local face
for l_f1 in xrange(0,4):
  # determine basis
  l_basisF1 = []
  for l_bf in xrange(0, len(l_bQuads)):
    if   l_f1 == 0: l_basisF1 = l_basisF1 + [ l_bQuads[l_bf].subs(eta, 0).subs( xi, chi)   ]
    elif l_f1 == 1: l_basisF1 = l_basisF1 + [ l_bQuads[l_bf].subs(xi, 1).subs( eta, chi)   ]
    elif l_f1 == 2: l_basisF1 = l_basisF1 + [ l_bQuads[l_bf].subs(eta, 1).subs(xi,  1-chi) ]
    elif l_f1 == 3: l_basisF1 = l_basisF1 + [ l_bQuads[l_bf].subs(xi, 0).subs( eta, 1-chi) ]

  # assemnle flux matrix
  l_localFm['fm'+str(l_f1)] = sympy.Matrix( l_basisF1 ) * \
                              sympy.Matrix.transpose( sympy.Matrix( l_basisF1 ))
    
  # do the integration
  l_localFm['fm'+str(l_f1)] = sympy.integrate( l_localFm['fm'+str(l_f1)], (chi, 0, 1) )

        
  # neighboring face
  for l_f2 in xrange(0,4):
    # basis at face, where chi is the shared integration parameter
    l_basisF2 = []
    
    # iterate over bfunctions
    for l_bf in xrange(0, len(l_bQuads)):
    if   l_f1 == 0: l_basisF1 = l_basisF1 + [ l_bQuads[l_bf].subs(eta, 0).subs( xi, 1-chi) ]
    elif l_f1 == 1: l_basisF1 = l_basisF1 + [ l_bQuads[l_bf].subs(xi, 1).subs( eta, 1-chi) ]
    elif l_f1 == 2: l_basisF1 = l_basisF1 + [ l_bQuads[l_bf].subs(eta, 1).subs(xi,    chi) ]
    elif l_f1 == 3: l_basisF1 = l_basisF1 + [ l_bQuads[l_bf].subs(xi, 0).subs( eta,   chi) ]

    # assemble the flux matrix
    l_neighFm['fp'+str(l_f1)+str(l_f2)] = sympy.Matrix( l_basisF2 ) * \
                                          sympy.Matrix.transpose( sympy.Matrix( l_basisF1 ))

    # do the integration
    l_neighFm['fp'+str(l_f1)+str(l_f2)] = sympy.integrate( l_neighFm['fp'+str(l_f1)+str(l_f2)], (chi, 0, 1) )

l_localFm['fm0'], l_localFm['fm1'], l_localFm['fm2'], l_localFm['fm3'], l_neighFm['fp33']

In [ ]:
l_localFm['fm0'], l_localFm['fm1'], l_localFm['fm2'], l_localFm['fm3'], l_neighFm['fp33']

In [ ]:
l_mmQuads, l_smQuads['xi'], l_smQuads['eta']

# Hexes
Remark: This is not a hierarchical format since we go for xi first and then for eta!

In [ ]:
# build quad basis functions as tensor product
l_bHexes = []
eta   = sympy.symbols('eta')
zeta  = sympy.symbols('zeta')
alpha = sympy.symbols('alpha')
beta  = sympy.symbols('beta')

for l_degZeta in xrange(0, l_degUp):
  for l_degEta in xrange(0, l_degUp):
    for l_degXi in xrange(0, l_degUp):
      l_bHexes = l_bHexes + [ l_pols[l_degXi] * l_pols[l_degEta].subs( xi, eta  ) * l_pols[l_degZeta].subs( xi, zeta) ]

In [ ]:
import mpmath
# compute the mass matrix
l_mmHexes = sympy.Matrix( l_bHexes ) * sympy.Matrix.transpose( sympy.Matrix( l_bHexes ) )

for l_en in xrange(0, len(l_mmHexes)):
  l_mmHexes[l_en] = sympy.integrate( l_mmHexes[l_en], (xi, 0, 1), (eta, 0, 1), (zeta, 0, 1) )

# compute the stiffness matrices
l_smHexes = {}
l_smHexes['xi']   = sympy.Matrix( l_bHexes )
l_smHexes['eta']  = sympy.Matrix( l_bHexes )
l_smHexes['zeta'] = sympy.Matrix( l_bHexes )

l_smHexes['xi']   = l_smHexes['xi']   * sympy.Matrix.transpose( sympy.Matrix( sympy.diff( l_smHexes['xi'],   xi   ) ) )
l_smHexes['eta']  = l_smHexes['eta']  * sympy.Matrix.transpose( sympy.Matrix( sympy.diff( l_smHexes['eta'],  eta  ) ) )
l_smHexes['zeta'] = l_smHexes['zeta'] * sympy.Matrix.transpose( sympy.Matrix( sympy.diff( l_smHexes['zeta'], zeta ) ) )

for l_en in xrange(0, len(l_smHexes['xi'])):
  l_smHexes['xi'][l_en]   = sympy.integrate( l_smHexes['xi'][l_en],   (xi, 0, 1), (eta, 0, 1), (zeta, 0, 1 ) )
  l_smHexes['eta'][l_en]  = sympy.integrate( l_smHexes['eta'][l_en],  (xi, 0, 1), (eta, 0, 1), (zeta, 0, 1 ) )
  l_smHexes['zeta'][l_en] = sympy.integrate( l_smHexes['zeta'][l_en], (xi, 0, 1), (eta, 0, 1), (zeta, 0, 1 ) )

In [ ]:
l_mmHexes, l_smHexes['zeta']

In [ ]:
# Here's the layout of our hexehedral elements:
#
#   face 0: 0-1-2-3
#   face 1: 0-1-5-4
#   face 2: 1-2-6-5
#   face 3: 3-2-6-7
#   face 4: 0-3-7-4
#   face 5: 4-5-6-7
#
#
#                     7 x*******************x 6
#                      **                 * *
#                     * *                 * *
#                    *  *                *  *
#                   *   *               *   *
#                  *    *              *    *
#               4 x*******************x 5   *
#                 *     *             *     *
#                 *   3 x************ * ****x 2  
#                 *    *              *    *
#                 *   *               *   *
#                 |  /                *  *
#            zeta | / eta             * *
#                 |/                  **
#                 x---****************x
#               0   xi                 1
#                                                     
# In the case of unstructured hexehedral meshes, 
# we have a total of 6 (local face) * 6 (neighboring face) * 4 (vertex mapping) = 64 flux matrices.
#
# The vertex mapping is unique by the position of the local face's first vertex to the local position
# in w.r.t. to the neighboring face's first vertex.
#
# An example:
#
# 5 ********* 6              4  ********* 7
#   *       *                   *       *
#   *   2   *     neighbors     *   4   *
#   *       *                   *       *
# 1 ********* 2               0 ********* 3 <--- location of local face 1
#
# ID:  2 * 6 * 4 (local face)
#    +     4 * 4 (neighboring face)
#    +         1 (location of vertex 1 is 3)
#
# We limit this derivation to rectangular (structured) meshes:
# This results in only six neighboring flux matrices since the mapping to the reference element
# does not involve any rotations:
#   * face 0 always neighbors face 5
#   * face 1 always neighbors face 3
#   * face 2 always neighbors face 4
#   * the first node of a local face always corresponds to the first node of the neighboring face

# compute flux matrices
l_localFm = dict()
l_neighFm = dict()

# determine basis
l_basisFlux = [[] for l_i in xrange(6)]
for l_bf in xrange(0, len(l_bHexes)):
  l_basisFlux[0] = l_basisFlux[0] + [ l_bHexes[l_bf].subs(zeta, 0).subs( xi,  alpha).subs( eta,  beta)  ]
  l_basisFlux[1] = l_basisFlux[1] + [ l_bHexes[l_bf].subs(eta,  0).subs( xi,  alpha).subs( zeta, beta) ]
  l_basisFlux[2] = l_basisFlux[2] + [ l_bHexes[l_bf].subs(xi,   1).subs( eta, alpha).subs( zeta, beta) ]
  l_basisFlux[3] = l_basisFlux[3] + [ l_bHexes[l_bf].subs(eta,  1).subs( xi,  alpha).subs( zeta, beta) ]
  l_basisFlux[4] = l_basisFlux[4] + [ l_bHexes[l_bf].subs(xi,   0).subs( eta, alpha).subs( zeta, beta) ]
  l_basisFlux[5] = l_basisFlux[5] + [ l_bHexes[l_bf].subs(zeta, 1).subs( xi,  alpha).subs( eta,  beta) ]

# assemble flux matrices
for l_f1 in xrange(0, 6):
  # assemnle flux matrix
  l_localFm['fm'+str(l_f1)] = sympy.Matrix( l_basisFlux[l_f1] ) * \
                              sympy.Matrix.transpose( sympy.Matrix( l_basisFlux[l_f1] ))
    
  # do the integration
  for l_en in xrange(0, len(l_smHexes['xi'])):
    l_localFm['fm'+str(l_f1)][l_en] = sympy.integrate( l_localFm['fm'+str(l_f1)][l_en], (alpha, 0, 1), (beta, 0, 1) )

        
  # neighboring face
  l_f2 = -1;
  if( l_f1 == 0 ): l_f2 = 5
  if( l_f1 == 1 ): l_f2 = 3
  if( l_f1 == 2 ): l_f2 = 4
  if( l_f1 == 3 ): l_f2 = 1
  if( l_f1 == 4 ): l_f2 = 2
  if( l_f1 == 5 ): l_f2 = 0
    
  # assemble flux matrix
  l_neighFm['fp'+str(l_f1)+str(l_f2)] = sympy.Matrix( l_basisFlux[l_f1] ) * \
                                        sympy.Matrix.transpose( sympy.Matrix( l_basisFlux[l_f2] ) )
    
  # do the integration
  for l_en in xrange(0, len(l_smHexes['xi'])):
    l_neighFm['fp'+str(l_f1)+str(l_f2)][l_en] = sympy.integrate( l_neighFm['fp'+str(l_f1)+str(l_f2)][l_en],\
                                                                 (alpha, 0, 1), (beta, 0, 1) )

In [ ]:
l_neighFm['fp50']